In [13]:
# 1. Importar las librerías necesarias
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# Importar TODAS las métricas necesarias
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sentence_transformers import SentenceTransformer

# 2. Carga y Limpieza
file_path = r"C:\\Users\\crist\\Desktop\\CursoEOI-IA\\Chapter-5\\Proyecto_Samsung\\Dataset\\cyberbullying_tweets_limpio_idioma.csv"
df4 = pd.read_csv(file_path)

# Seleccionar columnas y eliminar vacíos
df4 = df4[["texto_limpio", "cyberbullying_type"]].dropna()

# Limpiar texto (Regex)
def limpiar_texto(texto):
    texto = re.sub(r"@\w+", "", texto) 
    texto = re.sub(r"https?://\S+|www\.\S+", "", texto) 
    return texto

df4["texto_limpio"] = df4["texto_limpio"].apply(limpiar_texto)


In [14]:
# 'not_cyberbullying' = 0, Todo lo demás = 1
df4["cyberbullying_type"] = df4["cyberbullying_type"].apply(
    lambda x: 0 if str(x).strip() == "not_cyberbullying" else 1
)

In [ ]:
print(df4['texto_limpio'].str.count("@\w+").sum())
print(df4['texto_limpio'].str.count(r"https?://\S+|www\.\S+").sum())

0
0


<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\crist\AppData\Local\Temp\ipykernel_26276\1895982067.py:1: SyntaxWarning: invalid escape sequence '\w'
  print(df['texto_limpio'].str.count("@\w+").sum())


In [15]:
# Guardar el resultado en un nuevo archivo CSV
output_path = r"C:\\Users\\crist\\Desktop\\CursoEOI-IA\\Chapter-5\\Proyecto_Samsung\\Dataset\\tweets_sin_trad_limpios.csv"
df4.to_csv(output_path, index=False)
print(f"Archivo guardado en: {output_path}")

Archivo guardado en: C:\\Users\\crist\\Desktop\\CursoEOI-IA\\Chapter-5\\Proyecto_Samsung\\Dataset\\tweets_sin_trad_limpios.csv


In [16]:
# Instanciar el modelo multilingüe
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Generar los embeddings para el texto
X = model.encode(df4["texto_limpio"].tolist())
y = df4["cyberbullying_type"].values

# Dividir los datos en Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo de clasificación binaria (Logistic Regression)
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = clf.predict(X_test)

# Evaluar el modelo
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='binary') # 'binary' porque es 0 o 1
rec = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')

# Imprimir las métricas de evaluación
print("-" * 30)
print("RESULTADOS DEL MODELO SBERT MULTILINGÜE")
print("-" * 30)
print(f"Accuracy:  {acc:.4f}  (Porcentaje total de aciertos)")
print(f"Precision: {prec:.4f}  (Calidad de la detección)")
print(f"Recall:    {rec:.4f}  (Cantidad de bullying detectado)")
print(f"F1-score:  {f1:.4f}   (Balance entre Precision y Recall)")
print("-" * 30)

# El reporte detallado por clase también es muy útil
print("\nReporte detallado por clase:")
print(classification_report(y_test, y_pred, target_names=['No Bullying', 'Bullying']))

------------------------------
RESULTADOS DEL MODELO SBERT MULTILINGÜE
------------------------------
Accuracy:  0.8604  (Porcentaje total de aciertos)
Precision: 0.8891  (Calidad de la detección)
Recall:    0.9507  (Cantidad de bullying detectado)
F1-score:  0.9188   (Balance entre Precision y Recall)
------------------------------

Reporte detallado por clase:
              precision    recall  f1-score   support

 No Bullying       0.63      0.42      0.50      1602
    Bullying       0.89      0.95      0.92      7884

    accuracy                           0.86      9486
   macro avg       0.76      0.68      0.71      9486
weighted avg       0.85      0.86      0.85      9486



El dataset está desbalanceado (hay 5 veces más bullying que no-bullying). El modelo se ha vuelto "perezoso": ha aprendido que si dice "esto es bullying", acierta la mayoría de las veces.

In [18]:
# Instanciar el modelo multilingüe
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Generar los embeddings para el texto
X = model.encode(df4["texto_limpio"].tolist())
y = df4["cyberbullying_type"].values

# Dividir los datos en Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo de clasificación binaria (Logistic Regression)
# Modificamos el modelo para equilibrar los pesos de las clases automáticamente
clf = LogisticRegression(
    random_state=42, 
    max_iter=1000, 
    class_weight='balanced'  # <--- ESTA ES LA CLAVE
)

clf.fit(X_train, y_train)
# Realizar predicciones en el conjunto de prueba
y_pred = clf.predict(X_test)

# Evaluar el modelo
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='binary') # 'binary' porque es 0 o 1
rec = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')

# Imprimir las métricas de evaluación
print("-" * 30)
print("RESULTADOS DEL MODELO SBERT MULTILINGÜE")
print("-" * 30)
print(f"Accuracy:  {acc:.4f}  (Porcentaje total de aciertos)")
print(f"Precision: {prec:.4f}  (Calidad de la detección)")
print(f"Recall:    {rec:.4f}  (Cantidad de bullying detectado)")
print(f"F1-score:  {f1:.4f}   (Balance entre Precision y Recall)")
print("-" * 30)

# El reporte detallado por clase también es muy útil
print("\nReporte detallado por clase:")
print(classification_report(y_test, y_pred, target_names=['No Bullying', 'Bullying']))

------------------------------
RESULTADOS DEL MODELO SBERT MULTILINGÜE
------------------------------
Accuracy:  0.8003  (Porcentaje total de aciertos)
Precision: 0.9612  (Calidad de la detección)
Recall:    0.7917  (Cantidad de bullying detectado)
F1-score:  0.8683   (Balance entre Precision y Recall)
------------------------------

Reporte detallado por clase:
              precision    recall  f1-score   support

 No Bullying       0.45      0.84      0.59      1602
    Bullying       0.96      0.79      0.87      7884

    accuracy                           0.80      9486
   macro avg       0.71      0.82      0.73      9486
weighted avg       0.88      0.80      0.82      9486



| Métrica | Modelo ANTERIOR (Sin balancear) | Modelo ACTUAL (Balanceado) | Interpretación |
| :--- | :---: | :---: | :--- |
| **Recall (Bullying)** | 0.95 🚨 | 0.79 | El anterior detectaba casi todo, pero a costa de disparar a todo lo que se movía. |
| **Precision (Bullying)** | 0.89 | **0.96** ✅ | **Fiabilidad pura.** El nuevo modelo casi nunca acusa falsamente a un inocente. |
| **Recall (No Bullying)** | 0.42 ❌ | **0.84** ⭐ | **La gran mejora.** Ahora el sistema protege y reconoce correctamente a los usuarios inocentes. |
| **Accuracy Global** | 0.86 | 0.80 | Bajó ligeramente porque ahora el modelo es "justo" y no solo apuesta a la clase mayoritaria. |
| **Conclusión** | **Agresivo / Injusto** | **Robusto / Fiable** | El modelo actual es apto para producción real. |